In [2]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
import cv2
from tqdm import tqdm
import datetime
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dense, Reshape, Input, Add, Convolution2D, Flatten, Dropout, BatchNormalization, Conv2DTranspose, Activation, Lambda, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy,SparseCategoricalCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import clear_output, display, HTML
%matplotlib inline
from base64 import b64encode

## Loading the datasets

In [3]:
train_data_dir = "../data/one/training/image_2/"
train_gt_dir = "../data/one/training/gt_image_2/"
test_data_dir = "../data/one/testing/"

In [6]:
# Number of training examples
TRAINSET_SIZE = int(len(os.listdir(train_data_dir)) * 0.8)
print("Number of training examples:", TRAINSET_SIZE)

VALIDSET_SIZE = int(len(os.listdir(train_data_dir)) * 0.1)
print("Number of validation examples:", VALIDSET_SIZE)

TESTSET_SIZE = int(len(os.listdir(train_data_dir)) - TRAINSET_SIZE - VALIDSET_SIZE)
print("Number of test examples:", TESTSET_SIZE)

Number of training examples: 231
Number of validation examples: 28
Number of test examples: 30


In [7]:
# Initialize Constants
IMG_SIZE = 128
N_CHANNELS = 3
N_CLASSES = 1
SEED = 123

In [8]:
# function to load images and return a dictionary
def parse_image(image_path:str) -> dict:
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=N_CHANNELS)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  
  # The three types of img paths: um, umm, uu
  # gt image paths: um_road, umm_read, uu_read
  mask_path = tf.strings.regex_replace(image_path, "image_2", "gt_image_2")
  mask_path = tf.strings.regex_replace(mask_path, "um_", "um_road_")
  mask_path = tf.strings.regex_replace(mask_path, "umm_", "umm_road_")
  
  mask = tf.io.read_file(mask_path)
  mask = tf.image.decode_jpeg(mask, channels=N_CHANNELS)
  
  non_road_label = np.array([255,0,0])
  road_label = np.array([255,0,255])
  other_label = np.array([0,0,0])
  
  # convert mask to binary mask
  mask = tf.experimental.numpy.all(mask == road_label, axis=2)
  mask = tf.cast(mask, tf.uint8)
  mask = tf.expand_dims(mask, axis=-1)  # Add channel dimension
  
  
  return {'image':image, 'segmentation_mask':mask}

In [9]:
# Generate dataset variables
all_dataset = tf.data.Dataset.list_files(train_data_dir + "*.png", seed=SEED)
all_dataset = all_dataset.map(parse_image)

train_dataset = all_dataset.take(TRAINSET_SIZE + VALIDSET_SIZE)
val_dataset = train_dataset.skip(TRAINSET_SIZE)
train_dataset = train_dataset.take(TRAINSET_SIZE)
test_dataset = all_dataset.skip(TRAINSET_SIZE + VALIDSET_SIZE)

In [10]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(val_dataset))
print("Number of test examples:", len(test_dataset))

Number of training examples: 231
Number of validation examples: 28
Number of test examples: 30


## Applying transformation

In [11]:
# Tensorflow function to rescale images to [0, 1]
@tf.function
def normalize(input_image: tf.Tensor, input_mask: tf.Tensor) -> tuple:
  input_image = tf.cast(input_image, tf.float32) / 255.0
  return input_image, input_mask

# Tensorflow function to apply preprocessing transformations
@tf.function
def load_image_train(datapoint: dict) -> tuple:
  input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

# Tensorflow function to preprocess validation images
@tf.function
def load_image_test(datapoint: dict) -> tuple:
  input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [12]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000

dataset = {"train": train_dataset, "val": val_dataset, "test": test_dataset}

# -- Train Dataset --#
dataset['train'] = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.AUTOTUNE)
dataset['train'] = dataset['train'].shuffle(buffer_size=BUFFER_SIZE, seed=SEED)
dataset['train'] = dataset['train'].repeat()
dataset['train'] = dataset['train'].batch(BATCH_SIZE)
dataset['train'] = dataset['train'].prefetch(buffer_size=tf.data.AUTOTUNE)

#-- Validation Dataset --#
dataset['val'] = dataset['val'].map(load_image_test)
dataset['val'] = dataset['val'].repeat()
dataset['val'] = dataset['val'].batch(BATCH_SIZE)
dataset['val'] = dataset['val'].prefetch(buffer_size=tf.data.AUTOTUNE)

#-- Testing Dataset --#
dataset['test'] = dataset['test'].map(load_image_test)
dataset['test'] = dataset['test'].batch(BATCH_SIZE)
dataset['test'] = dataset['test'].prefetch(buffer_size=tf.data.AUTOTUNE)

print(dataset['train'])
print(dataset['val'])
print(dataset['test'])

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 128, 1), dtype=tf.float32, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 128, 1), dtype=tf.float32, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 128, 128, 1), dtype=tf.float32, name=None))>
